In [ ]:
import $ivy.`org.apache.spark::spark-sql:3.5.7`
import org.apache.log4j.{Level, Logger}
Logger.getLogger("org").setLevel(Level.OFF)
import org.apache.spark.sql._
import org.apache.spark.sql.types.{IntegerType, StringType, StructField, StructType}
import org.apache.spark.sql.functions._

val spark = SparkSession
                .builder()
                .master("local[*]")
                .appName("Dataframe API")
                .config("spark.log.level", "WARN")
                .getOrCreate()

import spark.implicits._

println(s"spark.version == ${spark.version}")